In [2]:
# install and load packages
import pandas as pd
import mechanize
from http.cookiejar import CookieJar
from bs4 import BeautifulSoup as soup
import html2text

In [3]:
# Load xlsx dataset
RotaryWing = pd.ExcelFile("..//Datasets/RotaryWing_021021_(modified).xlsx", engine='openpyxl')

In [33]:
# excel content
RotaryWing.sheet_names

['Sheet2', 'RW', 'RW Only', 'FlightAware']

In [4]:
# Extract "Data" sheet
RW_Only = RotaryWing.parse(2)

# Extract tail numbers
tail = RW_Only.TailNumber.to_list()

In [1]:
# login to the website
# Browser
br = mechanize.Browser()

# Cookie Jar
cj = CookieJar()
br.set_cookiejar(cj)

# Browser options
br.set_handle_equiv(True)
br.set_handle_gzip(True)
br.set_handle_redirect(True)
br.set_handle_referer(True)
br.set_handle_robots(False)
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

br.addheaders = [('User-agent', 'Chrome')]

# The site we will navigate into, handling it's session
br.open('https://flightaware.com/account/session')

# View available forms
for f in br.forms():
    print (f)

# Select the second (index one) form (the first form is a search query box)
br.select_form(nr=7)

# User credentials
br.form["flightaware_username"] = 'kismailo'
br.form['flightaware_password'] = 'gwynbleidd'

# Login
br.submit()

<get https://flightaware.com/account/locale_switch.rvt application/x-www-form-urlencoded
  <SelectControl(locale=[ar_SA, cs_CZ, de_DE, en_AU, en_CA, en_GB, *en_US, es_ES, es_MX, fi_FI, fr_CA, fr_FR, he_IL, hi_IN, hr_HR, it_IT, ja_JP, ko_KR, pl_PL, pt_BR, pt_PT, ru_RU, tr_TR, zh_CN, zh_HK, zh_TW])>>
<get https://flightaware.com/account/locale_switch.rvt application/x-www-form-urlencoded
  <SelectControl(locale=[ar_SA, cs_CZ, de_DE, en_AU, en_CA, en_GB, *en_US, es_ES, es_MX, fi_FI, fr_CA, fr_FR, he_IL, hi_IN, hr_HR, it_IT, ja_JP, ko_KR, pl_PL, pt_BR, pt_PT, ru_RU, tr_TR, zh_CN, zh_HK, zh_TW])>>
<get https://flightaware.com/ajax/ignoreall/omnisearch/disambiguation.rvt application/x-www-form-urlencoded
  <HiddenControl(searchterm=) (readonly)>
  <HiddenControl(token=) (readonly)>
  <HiddenControl(<None>=) (readonly)>>
<GET https://flightaware.com/live/findflight application/x-www-form-urlencoded
  <TextControl(origin=)>
  <TextControl(destination=)>
  <SubmitControl(<None>=) (readonly)>>
<

<response_seek_wrapper at 0x7f8bef8f57d0 whose wrapped object = <closeable_response at 0x7f8bf20e0d10 whose fp = <mechanize._gzip.UnzipWrapper object at 0x7f8bf20e0a10>>>

In [236]:
# tail number web scraping
def tail_numbers(tail):
    tailnumber = []
    dates = []
    aircraft = []
    origin = []
    destination = []
    departure = []
    arrival = []
    duration = []

    for t in tail:
        url = 'https://flightaware.com/live/flight/' + t + '/history/320'
        page_html = br.open(url).read()
        page_soup = soup(page_html, 'html.parser')
        table = page_soup.findAll("tr", {"class":["smallActiverow1 rowClickTarget", "smallActiverow2 rowClickTarget"]})

        for i in range(0,len(table)):
            tailnumber += [t]
            dates += table[i].find_all("td")[0].a
            aircraft += table[i].find_all("td")[1]
            origin += table[i].find_all("td")[2].span
            if len(table[i].find_all("td")[3]) != 0:
                destination += table[i].find_all("td")[3].span
            if len(table[i].find_all("td")[3]) == 0:
                destination += ["Unknown"]
            if len(table[i].find_all("td")[4]) == 1:
                departure += [table[i].find_all("td")[4].em]
            if len(table[i].find_all("td")[4]) == 2:
                departure += [table[i].find_all("td")[4]]
            if table[i].find_all("td")[5].em != 'NoneType':
                arrival += [table[i].find_all("td")[5]]
            if table[i].find_all("td")[5].em == 'NoneType':
                arrival += table[i].find_all("td")[5].em
            duration += table[i].find_all("td")[6]

    data = pd.DataFrame(list(zip(tailnumber, dates, aircraft, origin, destination, departure, arrival, duration)), 
                    columns=['tailnumber','dates', 'aircraft', 'origin', 'destination', 'departure', 'arrival', 'duration'])
    return(data)

In [225]:
# apply function
data = tail_numbers(tail)

In [ ]:
# merge names, and export data
data.merge(RW_Only[['TailNumber','NAME']], left_on='tailnumber', right_on='TailNumber', how='inner') \
    .to_csv(r'..//Datasets/RW_tail_numbers.csv', index=False)

In [ ]:
# BETA version
# tailnumber = []
# dates = []
# aircraft = []
# origin = []
# destination = []
# departure = []
# arrival = []
# duration = []

# for t in tail:
#     url = 'https://flightaware.com/live/flight/' + t + '/history/320'
#     page_html = br.open(url).read()
#     page_soup = soup(page_html, 'html.parser')
#     table = page_soup.findAll("tr", {"class":["smallActiverow1 rowClickTarget", "smallActiverow2 rowClickTarget"]})

#     for i in range(0,len(table)):
#         tailnumber += [t]
#         dates += table[i].find_all("td")[0].a
#         aircraft += table[i].find_all("td")[1]
#         origin += table[i].find_all("td")[2].span
#         destination += table[i].find_all("td")[3].span
#         if len(table[i].find_all("td")[4]) == 1:
#             departure += [table[i].find_all("td")[4].em]
#         if len(table[i].find_all("td")[4]) == 2:
#             departure += [table[i].find_all("td")[4]]
#         if len(table[i].find_all("td")[5]) == 1:
#             arrival += [table[i].find_all("td")[5].em]
#         if len(table[i].find_all("td")[5]) == 2:
#             arrival += [table[i].find_all("td")[5]]
#         duration += table[i].find_all("td")[6]

# data = pd.DataFrame(list(zip(tailnumber, dates, aircraft, origin, destination, departure, arrival, duration)), columns=['tailnumber','dates', 'aircraft', 'origin', 'destination', 'departure', 'arrival', 'duration'])